# 平均報酬強化学習

参考：
* [Average-reward model-free reinforcement learning: a systematic review and literature mapping](https://arxiv.org/abs/2010.08920)

今回は平均報酬強化学習について学びます．大体の証明はPuterman 1994にあるっぽい？

## 定常分布について

次を定義します：
* 方策から誘導されるマルコフ連鎖：$\boldsymbol{P}_\pi \in[0,1]^{|\mathcal{S}| \times|\mathcal{S}|}$
    * その$t$回繰り返したもの：$\boldsymbol{P}_\pi^t\left[s_0, s\right]=\operatorname{Pr}\left\{S_t=s \mid s_0, \pi\right\}=: p_\pi^t\left(s \mid s_0\right)$

このとき，このマルコフ連鎖の収束先は
$$p_\pi^{\star}\left(s \mid s_0\right)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} p_\pi^t\left(s \mid s_0\right)=\lim _{t_{\max } \rightarrow \infty} p_\pi^{t_{\max }}\left(s \mid s_0\right), \quad \forall s \in \mathcal{S}$$

を満たします．この最初のlimitは有限なMDPならば必ず存在し，２つ目のLimitはMDPがaperiodicであれば存在します（TODO: 証明）

また，$p_\pi^\star(s|s_0)$をベクトル化したものを$\boldsymbol{p}_\pi^{\star} \in[0,1]^{|\mathcal{S}|}$とします．
これは
$$\left(\boldsymbol{p}_\pi^{\star}\right)^{\top} \boldsymbol{P}_\pi=\left(\boldsymbol{p}_\pi^{\star}\right)^{\top}$$
を満たします

## ゲインについて

平均報酬はゲインとも呼ばれ，次で定義され，以下の変形が成立します：

$$
\begin{aligned}
v_g\left(\pi, s_0\right) & :=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \mathbb{E}_{S_t, A_t}\left[\sum_{t=0}^{t_{\max }-1} r\left(S_t, A_t\right) \mid S_0=s_0, \pi\right] \\
& =\sum_{s \in \mathcal{S}}\left\{\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} p_\pi^t\left(s \mid s_0\right)\right\} r_\pi(s)=\sum_{s \in \mathcal{S}} p_\pi^{\star}\left(s \mid s_0\right) r_\pi(s) \\
& =\lim _{t_{\max } \rightarrow \infty} \mathbb{E}_{S_{t_{\max }}, A_{t_{\max }}}\left[r\left(S_{t_{\max }}, A_{t_{\max }}\right) \mid S_0=s_0, \pi\right]
\end{aligned}
$$

ここで，
* $r_\pi(s)=\sum_{a \in \mathcal{A}} \pi(a \mid s) r(s, a)$
* 最初のlimitは方策が定常かつMDPが有限であれば存在します（TODO: 証明）
* ２つ目のLimitは簡単です
* ３つ目は方策が定常かつMDPがaperiodic，もしくは報酬に何らかの構造が入っている場合に存在します（TODO: 証明）

行列形式で書けば，

$$
\boldsymbol{v}_g(\pi)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \boldsymbol{v}_{t_{\max }}(\pi)=\lim _{t_{\max } \rightarrow \infty} \frac{1}{t_{\max }} \sum_{t=0}^{t_{\max }-1} \boldsymbol{P}_\pi^t \boldsymbol{r}_\pi=\boldsymbol{P}_\pi^{\star} \boldsymbol{r}_\pi
$$

です．ここで，$\boldsymbol{r}_\pi \in \mathbb{R}^{|\mathcal{S}|}$ は $r_\pi(s)$をstackしたものです．

定常分布は$s_0$に依存しないので，$\boldsymbol{P}_\pi^{\star}$の行は全て同じであることに気をつけましょう．つまり，

* $v_g(\pi)=\boldsymbol{p}_\pi^{\star} \cdot \boldsymbol{r}_\pi=v_g\left(\pi, s_0\right), \forall s_0 \in \mathcal{S}$
* $\boldsymbol{v}_g(\pi)=v_g(\pi) \cdot \mathbf{1}$

です．

方策$\pi^* \in \Pi_{\mathrm{S}}$は次を満たす時ゲイン最適であると呼びます：

$$
v_g\left(\pi^*, s_0\right) \geq v_g(\pi, s_0), \quad \forall \pi \in \Pi_{\mathrm{S}}, \forall s_0 \in \mathcal{S}, \quad \text { hence } \pi^* \in \underset{\pi \in \Pi_{\mathrm{S}}}{\operatorname{argmax}}\; v_g(\pi)
$$


## ベルマン方程式

平均報酬についても以下のベルマン方程式が成立します：

$$
v_b\left(\pi^*, s\right)+v_g\left(\pi^*\right)=\max _{a \in \mathcal{A}}\left\{r(s, a)+\sum_{c^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) v_b\left(\pi^*, s^{\prime}\right)\right\}, \quad \forall s \in \mathcal{S},
$$

こおｋで，$v_b$はバイアスと呼ばれ，次で定義されます：

$$
\begin{aligned}
v_b\left(\pi, s_0\right) & :=\lim _{t_{\max } \rightarrow \infty} \mathbb{E}_{S_t, A_t}\left[\sum_{t=0}^{t_{\max }-1}\left(r\left(S_t, A_t\right)-v_g(\pi)\right) \mid S_0=s_0, \pi\right] \\
& =\lim _{t_{\max } \rightarrow \infty} \sum_{t=0}^{t_{\max }-1} \sum_{s \in \mathcal{S}}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right) r_\pi(s) \\
& =\underbrace{\sum_{t=0}^{\tau-1} \sum_{s \in \mathcal{S}} p_\pi^t\left(s \mid s_0\right) r_\pi(s)}_{\text {the expected total reward } v_\tau}-\tau v_g(\pi)+\underbrace{\lim _{t_{\max } \rightarrow \infty} \sum_{t=\tau}^{t_{\max }-1} \sum_{s \in \mathcal{S}}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right) r_\pi(s)}_{\text {approaches } 0 \text { as } \tau \rightarrow \infty} \\
& =\sum_{s \in \mathcal{S}}\left\{\lim _{t_{\max } \rightarrow \infty} \sum_{t=0}^{t_{\max }-1}\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right)\right\} r_\pi(s)=\sum_{s \in \mathcal{S}} d_\pi\left(s \mid s_0\right) r_\pi(s),
\end{aligned}
$$

ここで，$d_\pi(s|s_0)$は行列$\boldsymbol{D}_\pi:=\left(\boldsymbol{I}-\boldsymbol{P}_\pi+\boldsymbol{P}_\pi^{\star}\right)^{-1}\left(\boldsymbol{I}-\boldsymbol{P}_\pi^{\star}\right)$と同値です（limitと期待値を入れ替えて良い場合）．

バイアスは以下の解釈ができます：

1. （１行目）$s_0$から始まって$\pi$に従うときの，即時報酬$r(s_t, a_t)$と定常報酬$v_g(\pi)$の差
2. （２行目）$s_0$から始まった過程と，$p_\pi^\star$からサンプルされた初期状態から始まった過程の，報酬の差．よって，$\pi$によって得られる総報酬と，ずっと$v_g(\pi)$の報酬の場合の総報酬の差．
3. （３行目）$v_\tau\left(\pi, s_0\right) \approx v_g(\pi) \tau+v_b\left(\pi, s_0\right)$とみなせるので，期待総報酬$v_\tau$がどれくらい振動するかを表している．
4. （４行目）$\left(p_\pi^t\left(s \mid s_0\right)-p_\pi^{\star}(s)\right)$は定常状態になるまで非ゼロなので，バイアスは一時的な性能とみなせる．

何らかの参照状態$s_{\mathrm{ref}} \in \mathcal{S}$について，相対価値$v_{\text {brel }}$が定義されます：

$$
v_{b r e l}(\pi, s):=v_b(\pi, s)-v_b\left(\pi, s_{\text {ref }}\right)=\lim _{t_{\max } \rightarrow \infty}\left\{v_{t_{\max }}(\pi, s)-v_{t_{\max }}\left(\pi, s_{\text {ref }}\right)\right\}, \quad \forall \pi \in \Pi_{\mathrm{S}}, \forall s \in \mathcal{S} .
$$


## 価値反復法

ベルマン方程式より，$\boldsymbol{v}_b^* \in \mathbb{R}^{\mid \mathcal{S}\mid}$がわかれば最適方策も求まります（このとき，最適ゲイン$v_g^*$も必要になります）．
そのため，目標は$\boldsymbol{v}_b^* \in \mathbb{R}^{\mid \mathcal{S}\mid}$を求めればよいです．
通常の動的計画法と同様にして，$\boldsymbol{v}_b^*$の代わりに$\boldsymbol{q}_b^*\in \mathbb{R}^{|\mathcal{S}||\mathcal{A}|}$を求めます．

---

**状態価値関数によるアプローチ**

1. $\hat{v}_b^{k=0}(s) \leftarrow 0$として初期化します
2. 適当な固定された参照状態$s_{\text{ref }}$を使って，次のベルマン作用素で更新します：$\hat{v}_b^{k+1}(s)=\mathbb{B}^*\left[\hat{v}_b^k(s)\right]-\hat{v}_b^k\left(s_{\text {ref }}\right), \quad \forall s \in \mathcal{S}$
$$
\mathbb{B}^*\left[\hat{v}_b^k(s)\right]:=\max _{a \in \mathcal{A}}\left[r(s, a)+\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) \hat{v}_b^k\left(s^{\prime}\right)\right], \quad \forall s \in \mathcal{S} .
$$
3. スパン関数を$s p(\boldsymbol{v}):=\max _{s \in \mathcal{S}} v(s)-\min _{s^{\prime} \in \mathcal{S}} v\left(s^{\prime}\right)$とします．$s p\left(\hat{\boldsymbol{v}}_b^{k+1}-\hat{\boldsymbol{v}}_b^k\right)\leq\varepsilon$ であればアルゴリズムを停止します．

このアルゴリズムは$k\to \infty$で$\hat{v}_b^k\left(s_{\text {ref }}\right)$が$v_g^*$に収束することが知られています．

---

**行動価値関数によるアプローチ**

行動価値関数を
$$
\begin{aligned}
q_b(\pi, s, a) & :=\lim _{t_{\max } \rightarrow \infty} \mathbb{E}_{S_t, A_t}\left[\sum_{t=0}^{t_{\max }-1}\left(r\left(S_t, A_t\right)-v_g^\pi\right) \mid S_0=s, A_0=a, \pi\right] \\
& =r(s, a)-v_g^\pi+\mathbb{E}\left[v_b^\pi\left(S_{t+1}\right)\right], \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A},
\end{aligned}
$$
で定義します．このとき，次のベルマン方程式が成立します．

$$
q_b^*(s, a)+v_g^*=r(s, a)+\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) \underbrace{\max _{a^{\prime} \in \mathcal{A}} q_b^*\left(s^{\prime}, a^{\prime}\right)}_{v_b^*\left(s^{\prime}\right)}, \quad \forall(s, a) \in \mathcal{S} \times \mathcal{A},
$$

これを使って，次によって$q$を更新することを考えます．

$$
\begin{aligned}
\hat{q}_b^{k+1}(s, a)&=\mathbb{B}^*\left[\hat{q}_b^k(s, a)\right]-v_b^k\left(s_{\text {ref }}\right)\\
&=\underbrace{r(s, a)+\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) \max _{a^{\prime} \in \mathcal{A}} \hat{q}_b^k\left(s^{\prime}, a^{\prime}\right)}_{\mathbb{B}^*\left[\hat{q}_b^k(s, a)\right]}-\underbrace{\max _{a^{\prime \prime} \in \mathcal{A}} \hat{q}_b^k\left(s_{\text {ref }}, a^{\prime \prime}\right)}_{\text {can be interpreted as } \hat{v}_g^*},
\end{aligned}
$$

この反復は$\hat{v}_b^k(s)=\max _{a \in \mathcal{A}} \hat{q}_b^k(s, a)$が$v_b^*(s)$に収束することが知られてます．

---

**方策評価**

次の方程式を解くことで方策評価ができます：（TODO: どうやって解くの？）

$$
v_b^k(s)+v_g^k=r(s, a)+\sum_{s^{\prime} \in \mathcal{S}} p\left(s^{\prime} \mid s, a\right) v_b^k\left(s^{\prime}\right), \quad \forall s \in \mathcal{S}, \text { with } a=\hat{\pi}^k(s)
$$

In [2]:
import numpy as np
import jax.numpy as jnp
from jax.random import PRNGKey
import jax
from typing import NamedTuple, Optional

key = PRNGKey(0)

S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合
gamma = 0.8  # 割引率


# 報酬行列を適当に作ります
key, _ = jax.random.split(key)
rew = jax.random.uniform(key=key, shape=(S, A))
assert rew.shape == (S, A)


# 遷移確率行列を適当に作ります
key, _ = jax.random.split(key)
P = jax.random.uniform(key=key, shape=(S*A, S))
P = P / jnp.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(S, A, S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class MDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    rew: jnp.array  # 報酬行列
    P: jnp.array  # 遷移確率行列

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


mdp = MDP(S_set, A_set, rew, P)

print("状態数：", mdp.S)
print("行動数：", mdp.A)

状態数： 10
行動数： 3


In [8]:

ref_state = 0
Q = jnp.zeros((S, A))

for i in range(30):
    next_v = mdp.P @ Q.max(axis=1)
    nQ = mdp.rew + next_v - Q[ref_state].max()
    span = (nQ.max(axis=1) - Q.max(axis=1)).max() - (nQ.max(axis=1) - Q.max(axis=1)).min()
    Q = nQ
    print(Q.max())

0.95673645
1.014119
1.0162408
1.0300635
1.0319318
1.0320619
1.0322145
1.0322174
1.0322226
1.0322232
1.0322232
1.0322232
1.0322232
1.0322232
1.0322231
1.0322233
1.0322232
1.0322231
1.0322233
1.0322232
1.0322232
1.0322232
1.0322232
1.0322232
1.0322232
1.0322232
1.0322231
1.0322233
1.0322232
1.0322231
